In [1]:
import pandas as pd
import numpy as np
import math
import math
from Util import *
from CLT_class import CLT
from __future__ import print_function

datasetList = ["pumsb_star","tretail"]
data = {}
for i in datasetList:
    data[i+".ts.data"] = Util.load_dataset("datasets\\dataset\\"+i+".ts.data")
    data[i+".test.data"] = Util.load_dataset("datasets\\dataset\\"+i+".test.data")
    data[i+".valid.data"] = Util.load_dataset("datasets\\dataset\\"+i+".valid.data")

    

In [9]:
class MIXTURE_CLT():
   
    def __init__(self):
        self.n_components = 0 # number of components
        self.mixture_probs = None # mixture probabilities
        self.clt_list =[]   # List of Tree Bayesian networks

    '''
        Learn Mixtures of Trees using the EM algorithm.
    '''
    def learn(self, dataset, n_components=2, max_iter=50, epsilon=format(1e-1, '.6f')):
        # For each component and each data point, we have a weight
        weights=np.random.random((n_components , dataset.shape[0]))
        self.mixture_probs = np.zeros(n_components)
 
        # Randomly initialize the chow-liu trees and the mixture probabilities
        # Your code for random initialization goes here
 
        for i in range(n_components):
            self.clt_list.append(CLT())
            self.clt_list[i].learn(dataset)
 
        a = np.random.rand(n_components)
        for i in range(len(a)):
            self.mixture_probs[i] = a[i] / np.sum(a)  

        like = 0.0
        old_like = 999
        prob = np.zeros((n_components , dataset.shape[0]))


        for itr in range(max_iter):
           
            print("Iteration - ",itr)
            print("N components - ",n_components)
           
            #E-step: Complete the dataset to yield a weighted dataset
            # We store the weights in an array weights[ncomponents,number of points]
            #Your code for E-step here
            
            if abs(old_like - like) <= epsilon:
                break


            for i in range(n_components):
                for j in range(dataset.shape[0]):
                    prob[i][j] = self.clt_list[i].getProb(dataset[j])
                weights[i] = np.multiply(self.mixture_probs[i], prob[i])/np.sum(np.multiply(self.mixture_probs[i], prob[i]))
                # print(weights.shape)
        
            # # M-step: Update the Chow-Liu Trees and the mixture probabilities
            # #Your code for M-Step here
            for i in range(n_components):
                self.clt_list[i].update(dataset, weights[i])
                self.mixture_probs[i] = weights[i].sum()/len(weights[i])
                
            old_like = like
            like = self.computeLL(dataset , n_components) / dataset.shape[0]
            print(like)

         
           
    # """
    #     Compute the log-likelihood score of the dataset
    # """
    def computeLL(self, dataset , n_components):
        ll = 0.0
        likelihood=0.0

        # Write your code below to compute likelihood of data
        #   Hint:   Likelihood of a data point "x" is sum_{c} P(c) T(x|c)
        #           where P(c) is mixture_prob of cth component and T(x|c) is the probability w.r.t. chow-liu tree at c
        #           To compute T(x|c) you can use the function given in class CLT
        for i in range (dataset.shape[0]):
            for j in range(n_components):
                likelihood=likelihood + self.mixture_probs[j]*self.clt_list[j].getProb(dataset[i])
            ll=ll+ np.log(likelihood)


        return ll
   
   

In [3]:
#MCLT
kval = {}
items = list(data.keys())
# mix_clt=MIXTURE_CLT.MIXTURE_CLT()
ncomponents=10 #number of components
max_iter=50 #max number of iterations for EM
epsilon=1e-2 #converge if the difference in the log-likelihods between two iterations is smaller 1e-1


i = 0

kvals = [2,5,10,20]
while i < 30:
    kMixCLT = [MIXTURE_CLT() for p in kvals]
    p = 0
    temp = []
    print("Running for dataset - ", items[i])
    for k in kvals:
        t = i 
        # print([items[t]])
        kMixCLT[p].learn(data[items[t]],k,max_iter,epsilon)
        t += 2
        x = kMixCLT[p].computeLL(data[items[t]] , k)/data[items[t]].shape[0]
        print("LL for ", k , " - ", x  )
        temp.append(x)
        p += 1
        
    kval[items[i]] = [ kvals[temp.index(max(temp))] , max(temp)] 

    print(temp)
    print(kval)
    i += 3

Running for dataset -  pumsb_star.ts.data
Iteration -  0
N components -  2
-11.00691165214752
Iteration -  1
N components -  2
-9.71731322284354
Iteration -  2
N components -  2
-8.847819606520764
Iteration -  3
N components -  2
-8.462277172892813
Iteration -  4
N components -  2
-8.369760780816172
Iteration -  5
N components -  2
-8.389251918408108
Iteration -  6
N components -  2
-8.455302359937438
Iteration -  7
N components -  2
-8.499509717785337
Iteration -  8
N components -  2
-8.593797923432064
Iteration -  9
N components -  2
-8.660800697212153
Iteration -  10
N components -  2
-8.720943325792216
Iteration -  11
N components -  2
-8.773684677239205
Iteration -  12
N components -  2
-8.818760135698195
Iteration -  13
N components -  2
-8.856459152144653
Iteration -  14
N components -  2
-8.887963263299024
Iteration -  15
N components -  2
-8.914688242482422
Iteration -  16
N components -  2
-8.937881361447328
Iteration -  17
N components -  2
-8.958441184963199
Iteration -  18

IndexError: list index out of range

In [ ]:
kval = {}
kval["pumsb_star.ts.data"] = [20, -17.298446571656125]
kval["tretail.ts.data"] = [20, -2.72441003063707]

In [10]:
# kval =["pumsb_star","tretail"]
items = list(data.keys())
ncomponents=20
max_iter=50 
epsilon=1e-1 
i = 0
temp = []

for k in kval.keys():
    for i in range(5):
        clt = MIXTURE_CLT()
        print("Testing for dataset - " , k)
        clt.learn(data[k+".ts.data"],20,max_iter,epsilon)
        x = clt.computeLL( data[k+".test.data"] , 20)/data[k+".test.data"].shape[0]
        print("LL for testing - " , x  )
        temp.append(x)
            

        print(temp)
        print(kval)

Testing for dataset -  pumsb_star
Iteration -  0
N components -  20
-8.704326559153861
Iteration -  1
N components -  20
-7.414728129849355
Iteration -  2
N components -  20
-6.5452345135269905
Iteration -  3
N components -  20
-6.159692079898852
Iteration -  4
N components -  20
-6.0671756878221155
Iteration -  5
N components -  20
LL for testing -  -12.668696522691103
[-12.668696522691103]
{'pumsb_star': [20, -17.298446571656125], 'tretail': [20, -2.72441003063707]}
Testing for dataset -  pumsb_star
Iteration -  0
N components -  20
-8.704326559153861
Iteration -  1
N components -  20
-7.4147281298493555
Iteration -  2
N components -  20
-6.5452345135269905
Iteration -  3
N components -  20
-6.159692079898852
Iteration -  4
N components -  20
-6.0671756878221155
Iteration -  5
N components -  20
LL for testing -  -12.664418899268348
[-12.668696522691103, -12.664418899268348]
{'pumsb_star': [20, -17.298446571656125], 'tretail': [20, -2.72441003063707]}
Testing for dataset -  pumsb_sta